<a href="https://colab.research.google.com/github/kimsoomin20221789/PyTorchExercise/blob/main/Lab06_Softmax_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rogistic Regression은 class가 두개(0 or 1) 로 분류되었습니다.
따라서 H(x)는 sigmoid를 사용하였고, loss Function은 BCE (Binary Cross Entropy)를 사용하였습니다.

Softmax Classification은 class 가 여러개로 분류됩니다.
따라서 H(x)는 softMax를 사용하고, loss Function은 CE (Cross Entropy)를 사용합니다.

In [1]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

# Softmax

In [4]:
z = torch.FloatTensor([1, 2, 3])

In [5]:
hypothesis = F.softmax(z, dim=0) # softmax 함수
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [6]:
hypothesis.sum()

tensor(1.)

# Cross Entropy

In [7]:
z = torch.rand(3, 5, requires_grad = True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [9]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([2, 1, 0])


In [10]:
y_one_hot = torch.zeros_like(hypothesis)

In [11]:
y_one_hot

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [13]:
# a.scatter_(dim, index, value)
# a 배열의 dim차원의 배열에서 index 자리에 value를 할당
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [16]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)
cost = F.nll_loss(F.log_softmax(z, dim=1), y)
print(cost)
cost = F.cross_entropy(z, y)
print(cost)

tensor(1.6682, grad_fn=<MeanBackward0>)
tensor(1.6682, grad_fn=<NllLossBackward0>)
tensor(1.6682, grad_fn=<NllLossBackward0>)


# Training with Low-level Cross Entrpoy Loss

In [18]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [17]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [25]:
optimizer = optim.SGD([W, b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # cost (CE) 계산 1
  hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
  y_one_hot = torch.zeros_like(hypothesis)
  y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
  cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1)

  # cost (CE) 계산 2
  z = x_train.matmul(W) + b
  cost = F.cross_entropy(z, y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch: {} / {} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch: 0 / 1000 Cost: 0.399290
Epoch: 100 / 1000 Cost: 0.365709
Epoch: 200 / 1000 Cost: 0.333016
Epoch: 300 / 1000 Cost: 0.306667
Epoch: 400 / 1000 Cost: 0.294459
Epoch: 500 / 1000 Cost: 0.285098
Epoch: 600 / 1000 Cost: 0.276318
Epoch: 700 / 1000 Cost: 0.268061
Epoch: 800 / 1000 Cost: 0.260277
Epoch: 900 / 1000 Cost: 0.252923
Epoch: 1000 / 1000 Cost: 0.245962


In [22]:
print(hypothesis)

tensor([[1.2586e-03, 3.0853e-02, 9.6789e-01],
        [1.5323e-02, 3.1147e-01, 6.7321e-01],
        [1.1016e-05, 4.6428e-01, 5.3571e-01],
        [1.6931e-05, 7.1985e-01, 2.8013e-01],
        [2.3945e-01, 7.2620e-01, 3.4348e-02],
        [1.0334e-01, 8.9665e-01, 1.5851e-06],
        [4.6349e-01, 5.3637e-01, 1.4025e-04],
        [6.0477e-01, 3.9522e-01, 1.3691e-05]], grad_fn=<SoftmaxBackward0>)


In [26]:
prediction = hypothesis >= torch.FloatTensor([0.5])

In [27]:
print(prediction)

tensor([[False, False,  True],
        [False, False,  True],
        [False, False,  True],
        [False,  True, False],
        [False,  True, False],
        [False,  True, False],
        [ True, False, False],
        [ True, False, False]])


# High-level Implementaion with nn.Module

In [32]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3) # 입력 차원이 4 (class 4) -> 출력 차원 3 (class 3)

  def forward(self, x):
    return self.linear(x)

In [33]:
model = SoftmaxClassifierModel()

In [34]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)
nb_epochs = 1000
for epoch in range(nb_epochs+1):
  prediction = model(x_train)
  cost = F.cross_entropy(prediction, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch%100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 4.661125
Epoch  100/1000 Cost: 0.711834
Epoch  200/1000 Cost: 0.630354
Epoch  300/1000 Cost: 0.573769
Epoch  400/1000 Cost: 0.523962
Epoch  500/1000 Cost: 0.476843
Epoch  600/1000 Cost: 0.430889
Epoch  700/1000 Cost: 0.385375
Epoch  800/1000 Cost: 0.339987
Epoch  900/1000 Cost: 0.295069
Epoch 1000/1000 Cost: 0.254579


In [35]:
hypothesis >= torch.FloatTensor([0.5])

tensor([[False, False,  True],
        [False, False,  True],
        [False, False,  True],
        [False,  True, False],
        [False,  True, False],
        [False,  True, False],
        [ True, False, False],
        [ True, False, False]])